<a href="https://colab.research.google.com/github/JuanZapa7a/semiotics/blob/main/larq1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pasos para Implementar y Comparar VGG8 Normal vs. Hardware-Aware en Larq

**Larq** es una librería diseñada para construir y entrenar redes neuronales binarizadas (BNNs) utilizando TensorFlow y Keras. Si estás interesado en realizar un entrenamiento consciente del hardware (considerando ruido, cuantización, etc.) para **VGG8** usando **Larq**, puedes lograrlo aprovechando las funcionalidades específicas de **Larq** para la binarización y entrenamiento de modelos compactos.